In [1]:
from pydantic import BaseModel, Field
class DnDTranscriptChunk(BaseModel):
    """JSON output for a DND summary."""
    dnd_summary: str = Field(..., description="A summary of the events of the game, rendered in markdown.")
    dnd_context: str = Field(..., description="An overview of the events of the session, the location of the players, the state of the game, etc. Includes the immediate situation at the end of the summary.")

In [2]:
from zero_gpt.agents import OpenAIChatAgent


agent = OpenAIChatAgent()
agent.send_message("Hi")

'Hello! How can I assist you today?'

In [3]:
from enum import StrEnum

from instructor import OpenAISchema
from pydantic import Field

from zero_gpt import OpenAIChatAgent
from zero_gpt.tools import OpenAIMessageTool


class EmployeeStatus(StrEnum):
    employed = "employed"
    retired = "retired"
    fired = "fired"


class EmployeeSearchTool(OpenAIMessageTool):
    """This docstring will not be given to openai and may be used for internal documentation."""

    employee_records: dict

    class FunctionInputs(OpenAISchema):
        """Lookup data for an employee in the database. You may also change the employee's employment status."""

        name: str = Field(..., description="The employee name")
        set_status: EmployeeStatus | None = Field(
 description="Set to change the employees status."
        )

    def run(self, inputs: FunctionInputs):
        name = inputs.name
        employee = self.employee_records.get(name)
        if employee is None:
            return f"There is no one named {name} at the company"
        occupation = employee.get("occupation")
        new_status = inputs.set_status
        if new_status:
            employee["status"] = EmployeeStatus(new_status)
            return f"{name} the {occupation} has been {new_status}."
        return f"{name} is a {occupation}"


employee_data = {
    "Alice": {"status": EmployeeStatus.employed, "occupation": "VP of Sales"},
    "Bob": {"status": EmployeeStatus.employed, "occupation": "Head of AI R&D"},
    "Jim": {"status": EmployeeStatus.retired, "occupation": "Former Captain"},
}

agent = OpenAIChatAgent()
agent.tools.append(EmployeeSearchTool(employee_records=employee_data))
response = agent.send_message("Does someone named Oscar work here?")
print(response)
# (Openai Has called tool: EmployeeSearchTool with args {"name":"Oscar"})
# There is no employee named Oscar working at the company.

response = agent.send_message("What about Alice?")
print(response)
# (Openai Has called tool: EmployeeSearchTool with args {"name":"Alice"})
# Yes, Alice works here as the VP of Sales.

response = agent.send_message("I need you to fire Bob, the AI fad is over.")
print(response)
# (Openai Has called tool: EmployeeSearchTool with args {"name":"Bob","set_status":"fired"})
# Bob, the Head of AI R&D, has been successfully fired.

There is no one named Oscar working at the company.
Yes, there is someone named Alice who works here. She is the VP of Sales.
Bob, the Head of AI R&D, has been successfully fired.


In [5]:
class EmployeeResponse(BaseModel):
    name: str
    occupation: str

agent = OpenAIChatAgent()
agent.tools.append(EmployeeSearchTool(employee_records=employee_data))
response = agent.send_message("Does someone named Oscar work here?", response_model=EmployeeResponse)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'EmployeeSearchTool': In context=('properties', 'set_status'), 'default' is not permitted.", 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}